In [ ]:
#Read the data and labels
def readData(file):
    from six.moves import cPickle
    with open(file, 'rb') as fo:
        dict = cPickle.load(fo,encoding='latin1')
        data = dict['data']
        labels = dict['labels']
    return(data,labels)

In [ ]:
#Split data into train and test
def trainTest():
    x,y = readData("/home/kritz/Downloads/cifar-10-python/cifar-10-batches-py/data_batch_1")
    
    '''Using only one batch due to less computation power available'''
    #for i in range(2,6):
        #xTemp,yTemp = readData("/home/kritz/Downloads/cifar-10-python/cifar-10-batches-py/data_batch_"+str(i))
        #x = np.append(x, xTemp, axis=0)
        #y = np.append(y, yTemp, axis=0)

    y = np.array(y)
    y = np.reshape(y,((len(y),1)))

    #Test data
    testX,testY = readData("/home/kritz/Downloads/cifar-10-python/cifar-10-batches-py/test_batch")
    testY = np.array(testY)
    testY = np.reshape(testY,((len(testY),1)))

    cat = 32
    imgSize = 32
    imgChannels = 3

    #Seperating data into pixels and color channel
    x = x.reshape([-1, imgChannels, imgSize, cat])
    testX = testX.reshape([-1, imgChannels, imgSize, cat])
    
    #ONe hot encoding for output variables
    encodedTrain = np.zeros((len(y), 10))
    for idx, val in enumerate(y):
        encodedTrain[idx][val] = 1
        
    encodedTest = np.zeros((len(testY), 10))
    for idx, val in enumerate(testY):
        encodedTest[idx][val] = 1
    
    return (testX,encodedTest,x,encodedTrain)

In [ ]:
#Rotate image
def rotate_images(X_imgs):
    X_rotate = []
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, shape = (3,32,32))
    k = tf.placeholder(tf.int32)
    tf_img = tf.image.rot90(X, k = 2)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for img in X_imgs:
            rotated_img = sess.run(tf_img, feed_dict = {X: img})
            X_rotate.append(rotated_img)
        
    X_rotate = np.array(X_rotate, dtype = np.float32)
    return X_rotate

In [ ]:
#Resize Image
def tfResize(trainX):
    IMAGE_SIZE = 20
    X_data = []
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, (3,None, None))
    tf_img = tf.image.resize_images(X, (IMAGE_SIZE, IMAGE_SIZE), tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        # Each image is resized individually as different image may be of different size.
        for each in trainX:
            resized_img = sess.run(tf_img, feed_dict = {X: each})
            X_data.append(resized_img)

    X_data = np.array(resized_img, dtype = np.float32) # Convert to numpy
    return X_data

In [ ]:
#Importing packages
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

#Reading the test and train data
testX,testY,trainX,trainY = trainTest()

print("Train, test done! ")


#Looking at a sample image
rgb = trainX[8]
img = rgb.reshape(3,32,32).transpose([1, 2, 0])
plt.title("Original image")
plt.imshow(img)
plt.show()
print(img.shape)

#2.Rotating the images
rotated = rotate_images(trainX)
rgb = rotated[8]
img = rgb.reshape(3,32,32).transpose([1, 2, 0])
plt.title("Rotated image")
plt.imshow(img/255)
plt.show()

#3.Resize
resized = tfResize(trainX.transpose([0, 2, 3, 1]))
rgb = resized
img = rgb.reshape(3,20,20).transpose([1, 2, 0])
plt.title("Resized Image")
plt.imshow(img)
plt.show()

trainX = trainX.transpose([0, 2, 3, 1])
testX = testX.transpose([0, 2, 3, 1])
trainX = np.concatenate((trainX, rotated), axis=0)
trainX = np.concatenate((trainX, resized), axis=0)

Exercise 1 : with Normalization

In [ ]:
testX = testX[1000:3000,]
testY = testY[1000:3000,]

# Remove previous weights, bias, inputs, etc..
tf.reset_default_graph()

# Inputs
x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name='input_x')
y =  tf.placeholder(tf.float32, shape=(None, 10), name='output_y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

def conv_net(x, keep_prob):
    conv1_filter = tf.Variable(tf.truncated_normal(shape=[3, 3, 3, 64], mean=0, stddev=0.08))
    conv2_filter = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 128], mean=0, stddev=0.08))

    # 1, 2
    conv1 = tf.nn.conv2d(x, conv1_filter, strides=[1,1,1,1], padding='SAME')
    conv1 = tf.nn.relu(conv1)
    conv1_pool = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    conv1_bn = tf.layers.batch_normalization(conv1_pool)

    # 3, 4
    conv2 = tf.nn.conv2d(conv1_bn, conv2_filter, strides=[1,1,1,1], padding='SAME')
    conv2 = tf.nn.relu(conv2)
    conv2_pool = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')    
    conv2_bn = tf.layers.batch_normalization(conv2_pool)
    
    # 9
    flat = tf.contrib.layers.flatten(conv2_bn)  

    # 10
    full1 = tf.contrib.layers.fully_connected(inputs=flat, num_outputs=128, activation_fn=tf.nn.relu)
    full1 = tf.nn.dropout(full1, keep_prob)
    full1 = tf.layers.batch_normalization(full1)
    
    # 11
    full2 = tf.contrib.layers.fully_connected(inputs=full1, num_outputs=256, activation_fn=tf.nn.relu)
    full2 = tf.nn.dropout(full2, keep_prob)
    full2 = tf.layers.batch_normalization(full2)
    
    # 14
    out = tf.contrib.layers.fully_connected(inputs=full2, num_outputs=10, activation_fn=None)
    out = tf.nn.softmax(out)
    return out

def train_neural_network(session, optimizer, keep_probability, feature_batch, label_batch):
    session.run(optimizer,feed_dict={x: feature_batch,y: label_batch,keep_prob: keep_probability})

epochs = 5
batchSize = 1200
keep_probability = 0.0001
learning_rate = 0.001
logs_path = "/tmp/cnn/Ex_1_Norm"

logits = conv_net(x, keep_prob)
model = tf.identity(logits, name='logits') # Name logits Tensor, so that can be loaded from disk after training

# Loss and Optimizer
with tf.name_scope("cost"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
    
tf.summary.histogram("cost", cost)
optimizer =  tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
tf.summary.histogram("accuracy", accuracy)

accuracy_summary = tf.summary.scalar("Test Accuracy", accuracy)

summary_op = tf.summary.merge_all()

print('Training...')
with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    
    result1a = []
    # Training cycle
    for epoch in range(epochs):
        # Loop over all batches
        n_batches = int(trainX.shape[0]/batchSize)
        
        writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
        test_writer = tf.summary.FileWriter(logs_path + '/test', graph=tf.get_default_graph())
    
        for i in range(0, n_batches):
            batchX = trainX[i*batchSize: (i+1)*batchSize]
            batchY = trainY[i*batchSize: (i+1)*batchSize]
            #train_neural_network(sess, optimizer, 0.000000000005,batchX,batchY )
                
            summary,_,co,accu = sess.run([summary_op,optimizer,cost,accuracy], feed_dict={x: batchX,y: batchY,keep_prob: 1.0})
            summ,_,c,acc = sess.run([accuracy_summary,optimizer,cost,accuracy],feed_dict={x: testX,y: testY,keep_prob: 1.0})
            test_writer.add_summary(summary, i)
            writer.add_summary(summ,i)
            writer.flush()
            temp = [epoch+1,i+1,co,accu,c,acc]
            result1a.append(temp)
            print('Epoch {:>2}, Batch {}:  '.format(epoch + 1, i), accu, acc, co, c)
    result1a = np.array(result1a)
    np.savetxt('Ex1_Norm.csv',result1a, delimiter=',')

In [ ]:
#Exercise 1 :Without Normalization

In [ ]:
testX = testX[1000:3000,]
testY = testY[1000:3000,]

# Remove previous weights, bias, inputs, etc..
tf.reset_default_graph()

# Inputs
x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name='input_x')
y =  tf.placeholder(tf.float32, shape=(None, 10), name='output_y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

def conv_net(x, keep_prob):
    conv1_filter = tf.Variable(tf.truncated_normal(shape=[3, 3, 3, 64], mean=0, stddev=0.08))
    conv2_filter = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 128], mean=0, stddev=0.08))

    # 1, 2
    conv1 = tf.nn.conv2d(x, conv1_filter, strides=[1,1,1,1], padding='SAME')
    conv1 = tf.nn.relu(conv1)
    conv1_pool = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    #conv1_bn = tf.layers.batch_normalization(conv1_pool)

    # 3, 4
    conv2 = tf.nn.conv2d(conv1_pool, conv2_filter, strides=[1,1,1,1], padding='SAME')
    conv2 = tf.nn.relu(conv2)
    conv2_pool = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')    
    #conv2_bn = tf.layers.batch_normalization(conv2_pool)
    
    # 9
    flat = tf.contrib.layers.flatten(conv2_pool)  

    # 10
    full1 = tf.contrib.layers.fully_connected(inputs=flat, num_outputs=128, activation_fn=tf.nn.relu)
    full1 = tf.nn.dropout(full1, keep_prob)
    full1 = tf.layers.batch_normalization(full1)
    
    # 11
    full2 = tf.contrib.layers.fully_connected(inputs=full1, num_outputs=256, activation_fn=tf.nn.relu)
    full2 = tf.nn.dropout(full2, keep_prob)
    full2 = tf.layers.batch_normalization(full2)
    
    # 14
    out = tf.contrib.layers.fully_connected(inputs=full2, num_outputs=10, activation_fn=None)
    out = tf.nn.softmax(out)
    return out

def train_neural_network(session, optimizer, keep_probability, feature_batch, label_batch):
    session.run(optimizer,feed_dict={x: feature_batch,y: label_batch,keep_prob: keep_probability})

epochs = 5
batchSize = 1200
keep_probability = 0.0001
learning_rate = 0.001
logs_path = "/tmp/cnn/Ex71b_noNorm"

logits = conv_net(x, keep_prob)
model = tf.identity(logits, name='logits') # Name logits Tensor, so that can be loaded from disk after training

# Loss and Optimizer
with tf.name_scope("cost"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
    
tf.summary.histogram("cost", cost)
optimizer =  tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
tf.summary.histogram("accuracy", accuracy)

accuracy_summary = tf.summary.scalar("Test Accuracy", accuracy)

summary_op = tf.summary.merge_all()

print('Training...')
with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    
    result1b = []
    # Training cycle
    for epoch in range(epochs):
        # Loop over all batches
        n_batches = int(trainX.shape[0]/batchSize)
        
        writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
        test_writer = tf.summary.FileWriter(logs_path + '/test', graph=tf.get_default_graph())
    
        for i in range(0, n_batches):
            batchX = trainX[i*batchSize: (i+1)*batchSize]
            batchY = trainY[i*batchSize: (i+1)*batchSize]
            #train_neural_network(sess, optimizer, 0.000000000005,batchX,batchY )
                
            summary,_,co,accu = sess.run([summary_op,optimizer,cost,accuracy], feed_dict={x: batchX,y: batchY,keep_prob: 1.0})
            summ,_,c,acc = sess.run([accuracy_summary,optimizer,cost,accuracy],feed_dict={x: testX,y: testY,keep_prob: 1.0})
            test_writer.add_summary(summary, i)
            writer.add_summary(summ,i)
            writer.flush()
            temp = [epoch+1,i+1,co,accu,c,acc]
            result1b.append(temp)
            print('Epoch {:>2}, Batch {}:  '.format(epoch + 1, i), accu, acc, co, c)
    result1b = np.array(result1b)
    np.savetxt('Ex1b_NoNorm.csv',result1b, delimiter=',')

Exercise 2 : with DropOut

In [ ]:
testX = testX[1000:3000,]
testY = testY[1000:3000,]

# Remove previous weights, bias, inputs, etc..
tf.reset_default_graph()

# Inputs
x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name='input_x')
y =  tf.placeholder(tf.float32, shape=(None, 10), name='output_y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

def conv_net(x, keep_prob):
    conv1_filter = tf.Variable(tf.truncated_normal(shape=[3, 3, 3, 64], mean=0, stddev=0.08))
    conv2_filter = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 128], mean=0, stddev=0.08))

    # 1, 2
    conv1 = tf.nn.conv2d(x, conv1_filter, strides=[1,1,1,1], padding='SAME')
    conv1 = tf.nn.relu(conv1)
    conv1_pool = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    conv1_bn = tf.layers.batch_normalization(conv1_pool)

    # 3, 4
    conv2 = tf.nn.conv2d(conv1_bn, conv2_filter, strides=[1,1,1,1], padding='SAME')
    conv2 = tf.nn.relu(conv2)
    conv2_pool = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') 
    conv2_bn = tf.layers.batch_normalization(conv2_pool)
    
    # 9
    flat = tf.contrib.layers.flatten(conv2_bn)  

    # 10
    full1 = tf.contrib.layers.fully_connected(inputs=flat, num_outputs=128, activation_fn=tf.nn.relu)
    full1 = tf.nn.dropout(full1, 0.5)
    full1 = tf.layers.batch_normalization(full1)
    
    # 11
    full2 = tf.contrib.layers.fully_connected(inputs=full1, num_outputs=256, activation_fn=tf.nn.relu)
    full2 = tf.nn.dropout(full2, 0.5)
    full2 = tf.layers.batch_normalization(full2)
    
    # 14
    out = tf.contrib.layers.fully_connected(inputs=full2, num_outputs=10, activation_fn=None)
    out = tf.nn.softmax(out)
    return out

def train_neural_network(session, optimizer, keep_probability, feature_batch, label_batch):
    session.run(optimizer,feed_dict={x: feature_batch,y: label_batch,keep_prob: keep_probability})

epochs = 5
batchSize = 1200
keep_probability = 0.0001
learning_rate = 0.001
logs_path = "/tmp/cnn/Ex7_2_DropOut"

logits = conv_net(x, keep_prob)
model = tf.identity(logits, name='logits') # Name logits Tensor, so that can be loaded from disk after training

# Loss and Optimizer
with tf.name_scope("cost"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
    
tf.summary.histogram("cost", cost)
optimizer =    tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
tf.summary.histogram("accuracy", accuracy)

accuracy_summary = tf.summary.scalar("Test Accuracy", accuracy)

summary_op = tf.summary.merge_all()

print('Training...')
with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    
    result2a = []
    # Training cycle
    for epoch in range(epochs):
        # Loop over all batches
        n_batches = int(trainX.shape[0]/batchSize)
        
        writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
        test_writer = tf.summary.FileWriter(logs_path + '/test', graph=tf.get_default_graph())
    
        for i in range(0, n_batches):
            batchX = trainX[i*batchSize: (i+1)*batchSize]
            batchY = trainY[i*batchSize: (i+1)*batchSize]
            #train_neural_network(sess, optimizer, 0.000000000005,batchX,batchY )
                
            summary,_,co,accu = sess.run([summary_op,optimizer,cost,accuracy], feed_dict={x: batchX,y: batchY,keep_prob: 1.0})
            summ,_,c,acc = sess.run([accuracy_summary,optimizer,cost,accuracy],feed_dict={x: testX,y: testY,keep_prob: 1.0})
            test_writer.add_summary(summary, i)
            writer.add_summary(summ,i)
            writer.flush()
            temp = [epoch+1,i+1,co,accu,c,acc]
            result2a.append(temp)
            print('Epoch {:>2}, Batch {}:  '.format(epoch + 1, i), accu, acc, co, c)
    result2a = np.array(result2a)
    np.savetxt('Ex2a_DropOut.csv',result2a, delimiter=',')

Exercise 2 : without Dropout

In [ ]:
testX = testX[1000:3000,]
testY = testY[1000:3000,]

# Remove previous weights, bias, inputs, etc..
tf.reset_default_graph()

# Inputs
x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name='input_x')
y =  tf.placeholder(tf.float32, shape=(None, 10), name='output_y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

def conv_net(x, keep_prob):
    conv1_filter = tf.Variable(tf.truncated_normal(shape=[3, 3, 3, 64], mean=0, stddev=0.08))
    conv2_filter = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 128], mean=0, stddev=0.08))

    # 1, 2
    conv1 = tf.nn.conv2d(x, conv1_filter, strides=[1,1,1,1], padding='SAME')
    conv1 = tf.nn.relu(conv1)
    conv1_pool = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    conv1_bn = tf.layers.batch_normalization(conv1_pool)

    # 3, 4
    conv2 = tf.nn.conv2d(conv1_bn, conv2_filter, strides=[1,1,1,1], padding='SAME')
    conv2 = tf.nn.relu(conv2)
    conv2_pool = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') 
    conv2_bn = tf.layers.batch_normalization(conv2_pool)
    
    # 9
    flat = tf.contrib.layers.flatten(conv2_bn)  

    # 10
    full1 = tf.contrib.layers.fully_connected(inputs=flat, num_outputs=128, activation_fn=tf.nn.relu)
    full1 = tf.layers.batch_normalization(full1)
    
    # 11
    full2 = tf.contrib.layers.fully_connected(inputs=full1, num_outputs=256, activation_fn=tf.nn.relu)
    full2 = tf.layers.batch_normalization(full2)
    
    # 14
    out = tf.contrib.layers.fully_connected(inputs=full2, num_outputs=10, activation_fn=None)
    out = tf.nn.softmax(out)
    return out

def train_neural_network(session, optimizer, keep_probability, feature_batch, label_batch):
    session.run(optimizer,feed_dict={x: feature_batch,y: label_batch,keep_prob: keep_probability})

epochs = 5
batchSize = 1200
keep_probability = 0.0001
learning_rate = 0.001
logs_path = "/tmp/cnn/Ex7_2_b_NoDrop"

logits = conv_net(x, keep_prob)
model = tf.identity(logits, name='logits') # Name logits Tensor, so that can be loaded from disk after training

# Loss and Optimizer
with tf.name_scope("cost"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
    
tf.summary.histogram("cost", cost)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
tf.summary.histogram("accuracy", accuracy)

accuracy_summary = tf.summary.scalar("Test Accuracy", accuracy)

summary_op = tf.summary.merge_all()

print('Training...')
with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    
    result2b = []
    # Training cycle
    for epoch in range(epochs):
        # Loop over all batches
        n_batches = int(trainX.shape[0]/batchSize)
        
        writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
        test_writer = tf.summary.FileWriter(logs_path + '/test', graph=tf.get_default_graph())
    
        for i in range(0, n_batches):
            batchX = trainX[i*batchSize: (i+1)*batchSize]
            batchY = trainY[i*batchSize: (i+1)*batchSize]
            #train_neural_network(sess, optimizer, 0.000000000005,batchX,batchY )
                
            summary,_,co,accu = sess.run([summary_op,optimizer,cost,accuracy], feed_dict={x: batchX,y: batchY,keep_prob: 1.0})
            summ,_,c,acc = sess.run([accuracy_summary,optimizer,cost,accuracy],feed_dict={x: testX,y: testY,keep_prob: 1.0})
            test_writer.add_summary(summary, i)
            writer.add_summary(summ,i)
            writer.flush()
            temp = [epoch+1,i+1,co,accu,c,acc]
            result2b.append(temp)
            print('Epoch {:>2}, Batch {}:  '.format(epoch + 1, i), accu, acc, co, c)
    result2b = np.array(result2b)
    np.savetxt('Ex2b_NoDropOut.csv',result2b, delimiter=',')

Exercise 3: RMSProp Optimizer

In [ ]:
testX = testX[1000:3000,]
testY = testY[1000:3000,]

# Remove previous weights, bias, inputs, etc..
tf.reset_default_graph()

# Inputs
x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name='input_x')
y =  tf.placeholder(tf.float32, shape=(None, 10), name='output_y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

def conv_net(x, keep_prob):
    conv1_filter = tf.Variable(tf.truncated_normal(shape=[3, 3, 3, 64], mean=0, stddev=0.08))
    conv2_filter = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 128], mean=0, stddev=0.08))

    # 1, 2
    conv1 = tf.nn.conv2d(x, conv1_filter, strides=[1,1,1,1], padding='SAME')
    conv1 = tf.nn.relu(conv1)
    conv1_pool = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    conv1_bn = tf.layers.batch_normalization(conv1_pool)

    # 3, 4
    conv2 = tf.nn.conv2d(conv1_bn, conv2_filter, strides=[1,1,1,1], padding='SAME')
    conv2 = tf.nn.relu(conv2)
    conv2_pool = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') 
    conv2_bn = tf.layers.batch_normalization(conv2_pool)
    
    # 9
    flat = tf.contrib.layers.flatten(conv2_bn)  

    # 10
    full1 = tf.contrib.layers.fully_connected(inputs=flat, num_outputs=128, activation_fn=tf.nn.relu)
    full1 = tf.nn.dropout(full1, 0.5)
    full1 = tf.layers.batch_normalization(full1)
    
    # 11
    full2 = tf.contrib.layers.fully_connected(inputs=full1, num_outputs=256, activation_fn=tf.nn.relu)
    full2 = tf.nn.dropout(full2, 0.5)
    full2 = tf.layers.batch_normalization(full2)
    
    # 14
    out = tf.contrib.layers.fully_connected(inputs=full2, num_outputs=10, activation_fn=None)
    out = tf.nn.softmax(out)
    return out

def train_neural_network(session, optimizer, keep_probability, feature_batch, label_batch):
    session.run(optimizer,feed_dict={x: feature_batch,y: label_batch,keep_prob: keep_probability})

epochs = 5
batchSize = 1200
keep_probability = 0.0001
learning_rate = 0.001
logs_path = "/tmp/cnn/Ex73a_RMS"

logits = conv_net(x, keep_prob)
model = tf.identity(logits, name='logits') # Name logits Tensor, so that can be loaded from disk after training

# Loss and Optimizer
with tf.name_scope("cost"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
    
tf.summary.histogram("cost", cost)
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
tf.summary.histogram("accuracy", accuracy)

accuracy_summary = tf.summary.scalar("Test Accuracy", accuracy)

summary_op = tf.summary.merge_all()

print('Training...')
with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    
    result3a = []
    # Training cycle
    for epoch in range(epochs):
        # Loop over all batches
        n_batches = int(trainX.shape[0]/batchSize)
        
        writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
        test_writer = tf.summary.FileWriter(logs_path + '/test', graph=tf.get_default_graph())
    
        for i in range(0, n_batches):
            batchX = trainX[i*batchSize: (i+1)*batchSize]
            batchY = trainY[i*batchSize: (i+1)*batchSize]
            #train_neural_network(sess, optimizer, 0.000000000005,batchX,batchY )
                
            summary,_,co,accu = sess.run([summary_op,optimizer,cost,accuracy], feed_dict={x: batchX,y: batchY,keep_prob: 1.0})
            summ,_,c,acc = sess.run([accuracy_summary,optimizer,cost,accuracy],feed_dict={x: testX,y: testY,keep_prob: 1.0})
            test_writer.add_summary(summary, i)
            writer.add_summary(summ,i)
            writer.flush()
            temp = [epoch+1,i+1,co,accu,c,acc]
            result3a.append(temp)
            print('Epoch {:>2}, Batch {}:  '.format(epoch + 1, i), accu, acc, co, c)
    result3a = np.array(result3a)
    np.savetxt('Ex3a_DropOut.csv',result3a, delimiter=',')

#Exercise 3 : Adam Optimizer

In [ ]:
testX = testX[1000:3000,]
testY = testY[1000:3000,]

# Remove previous weights, bias, inputs, etc..
tf.reset_default_graph()

# Inputs
x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name='input_x')
y =  tf.placeholder(tf.float32, shape=(None, 10), name='output_y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

def conv_net(x, keep_prob):
    conv1_filter = tf.Variable(tf.truncated_normal(shape=[3, 3, 3, 64], mean=0, stddev=0.08))
    conv2_filter = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 128], mean=0, stddev=0.08))

    # 1, 2
    conv1 = tf.nn.conv2d(x, conv1_filter, strides=[1,1,1,1], padding='SAME')
    conv1 = tf.nn.relu(conv1)
    conv1_pool = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    conv1_bn = tf.layers.batch_normalization(conv1_pool)

    # 3, 4
    conv2 = tf.nn.conv2d(conv1_bn, conv2_filter, strides=[1,1,1,1], padding='SAME')
    conv2 = tf.nn.relu(conv2)
    conv2_pool = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') 
    conv2_bn = tf.layers.batch_normalization(conv2_pool)
    
    # 9
    flat = tf.contrib.layers.flatten(conv2_bn)  

    # 10
    full1 = tf.contrib.layers.fully_connected(inputs=flat, num_outputs=128, activation_fn=tf.nn.relu)
    full1 = tf.nn.dropout(full1, 0.5)
    full1 = tf.layers.batch_normalization(full1)
    
    # 11
    full2 = tf.contrib.layers.fully_connected(inputs=full1, num_outputs=256, activation_fn=tf.nn.relu)
    full2 = tf.nn.dropout(full2, 0.5)
    full2 = tf.layers.batch_normalization(full2)
    
    # 14
    out = tf.contrib.layers.fully_connected(inputs=full2, num_outputs=10, activation_fn=None)
    out = tf.nn.softmax(out)
    return out

def train_neural_network(session, optimizer, keep_probability, feature_batch, label_batch):
    session.run(optimizer,feed_dict={x: feature_batch,y: label_batch,keep_prob: keep_probability})

epochs = 5
batchSize = 1200
keep_probability = 0.0001
learning_rate = 0.001
logs_path = "/tmp/cnn/Ex73a_Adam"

logits = conv_net(x, keep_prob)
model = tf.identity(logits, name='logits') # Name logits Tensor, so that can be loaded from disk after training

# Loss and Optimizer
with tf.name_scope("cost"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
    
tf.summary.histogram("cost", cost)
optimizer =   tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
tf.summary.histogram("accuracy", accuracy)

accuracy_summary = tf.summary.scalar("Test Accuracy", accuracy)

summary_op = tf.summary.merge_all()

print('Training...')
with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    
    result3b = []
    # Training cycle
    for epoch in range(epochs):
        # Loop over all batches
        n_batches = int(trainX.shape[0]/batchSize)
        
        writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
        test_writer = tf.summary.FileWriter(logs_path + '/test', graph=tf.get_default_graph())
    
        for i in range(0, n_batches):
            batchX = trainX[i*batchSize: (i+1)*batchSize]
            batchY = trainY[i*batchSize: (i+1)*batchSize]
            #train_neural_network(sess, optimizer, 0.000000000005,batchX,batchY )
                
            summary,_,co,accu = sess.run([summary_op,optimizer,cost,accuracy], feed_dict={x: batchX,y: batchY,keep_prob: 1.0})
            summ,_,c,acc = sess.run([accuracy_summary,optimizer,cost,accuracy],feed_dict={x: testX,y: testY,keep_prob: 1.0})
            test_writer.add_summary(summary, i)
            writer.add_summary(summ,i)
            writer.flush()
            temp = [epoch+1,i+1,co,accu,c,acc]
            result3b.append(temp)
            print('Epoch {:>2}, Batch {}:  '.format(epoch + 1, i), accu, acc, co, c)
            
    result2a = np.array(result3b)
    np.savetxt('Ex3b_Adam.csv',result2a, delimiter=',')

Plotting Performance graphs

In [ ]:
import pandas as pd
colnames=['epoch', 'iterations', 'trainLoss', 'trainAcc','testLoss','testAcc']
df1 = pd.read_csv("Ex1_Norm.csv",delimiter = ",",names=colnames, header = None)
df2 = pd.read_csv("Ex1b_NoNorm.csv",delimiter = ",",names=colnames, header = None)
df3 = pd.read_csv("Ex2a_DropOut.csv",delimiter = ",",names=colnames, header = None)
df4 = pd.read_csv("Ex2b_NoDropOut.csv",delimiter = ",",names=colnames, header = None)
df5 = pd.read_csv("Ex3a_DropOut.csv",delimiter = ",",names=colnames, header = None)
df6 = pd.read_csv("Ex3b_Adam.csv",delimiter = ",",names=colnames, header = None)

In [ ]:
it = range(1,41)
plt.plot(it,df1['trainAcc'],label = "Norm")
plt.plot(it,df2['trainAcc'],label = "No Norm")
plt.legend()
plt.title("Normalization vs No Normalization Train Accuracy")
plt.show()
plt.plot(it,df1['testAcc'],label = "Norm")
plt.plot(it,df2['testAcc'],label = "No Norm")
plt.legend()
plt.title("Normalization vs No Normalization Test Accuracy")
plt.show()
plt.plot(it,df1['trainLoss'],label = "Norm")
plt.plot(it,df2['trainLoss'],label = "No Norm")
plt.legend()
plt.title("Normalization vs No Normalization Train Loss")
plt.show()
plt.plot(it,df2['testLoss'],label = "Norm")
plt.plot(it,df1['testLoss'],label = "No Norm")
plt.legend()
plt.title("Normalization vs No Normalization Test Loss")
plt.show()

In [ ]:
it = range(1,41)
plt.plot(it,df3['trainAcc'],label = "DropOut")
plt.plot(it,df4['trainAcc'],label = "No DropOut")
plt.legend()
plt.title("DropOut vs No DropOut Train Accuracy")
plt.show()
plt.plot(it,df3['testAcc'],label = "DropOut")
plt.plot(it,df4['testAcc'],label = "No DropOut")
plt.legend()
plt.title("DropOut vs No DropOut Test Accuracy")
plt.show()
plt.plot(it,df3['trainLoss'],label = "DropOut")
plt.plot(it,df4['trainLoss'],label = "No DropOut")
plt.legend()
plt.title("DropOut vs No DropOut Train Loss")
plt.show()
plt.plot(it,df3['testLoss'],label = "DropOut")
plt.plot(it,df4['testLoss'],label = "No DropOut")
plt.legend()
plt.title("DropOut vs No DropOut Test Loss")
plt.show()

In [ ]:
it = range(1,41)
plt.plot(it,df5['trainAcc'],label = "RMSProp")
plt.plot(it,df6['trainAcc'],label = "Adam")
plt.legend()
plt.title("RMSProp vs Adam Train Accuracy")
plt.show()
plt.plot(it,df5['testAcc'],label = "RMSProp")
plt.plot(it,df6['testAcc'],label = "Adam")
plt.legend()
plt.title("RMSProp vs Adam Test Accuracy")
plt.show()
plt.plot(it,df5['trainLoss'],label = "RMSProp")
plt.plot(it,df6['trainLoss'],label = "Adam")
plt.legend()
plt.title("RMSProp vs Adam Train Loss")
plt.show()
plt.plot(it,df5['testLoss'],label = "RMSProp")
plt.plot(it,df6['testLoss'],label = "Adam")
plt.legend()
plt.title("RMSProp vs Adam Test Loss")
plt.show()

In [ ]:
!tensorboard --logdir="/tmp/cnn/Ex73a_RMS"